In [6]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager
import time 
import random

In [2]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - About to download new driver from https://chromedriver.storage.googleapis.com/103.0.5060.53/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\Rob\.wdm\drivers\chromedriver\win32\103.0.5060.53]


In [140]:
# url_list = ['https://www.cargurus.com/Cars/new/searchresults.action?zip=76012&showNegotiable=true&sortDir=ASC&sourceContext=homePageNewCarTab_false_0&distance=50&sortType=PRICE&entitySelectingHelper.selectedEntity=d320#listing=333784109/NEWCAR_FEATURED',
#            'https://www.cargurus.com/Cars/new/searchresults.action?zip=76012&showNegotiable=true&sortDir=ASC&sourceContext=homePageNewCarTab_false_0&distance=50&sortType=PRICE&entitySelectingHelper.selectedEntity=d320#listing=333784118/NEWCAR_FEATURED',
#            'https://www.cargurus.com/Cars/new/searchresults.action?zip=78704&showNegotiable=true&sortDir=ASC&sourceContext=homePageNewCarTab_false_0&distance=50&sortType=PRICE&entitySelectingHelper.selectedEntity=d2412#listing=333651233/NONE',
#            'https://www.cargurus.com/Cars/new/searchresults.action?zip=76012&showNegotiable=true&sortDir=ASC&sourceContext=homePageNewCarTab_false_0&distance=50&sortType=PRICE&entitySelectingHelper.selectedEntity=d320#listing=333784109/NEWCAR_FEATURED']

url_df = pd.read_csv('../Resources/all_urls_list.csv')
url_list = url_df['urls']
url_list

0       https://www.cargurus.com/Cars/new/searchresult...
1       https://www.cargurus.com/Cars/new/searchresult...
2       https://www.cargurus.com/Cars/new/searchresult...
3       https://www.cargurus.com/Cars/new/searchresult...
4       https://www.cargurus.com/Cars/new/searchresult...
                              ...                        
2216    https://www.cargurus.com/Cars/new/searchresult...
2217    https://www.cargurus.com/Cars/new/searchresult...
2218    https://www.cargurus.com/Cars/new/searchresult...
2219    https://www.cargurus.com/Cars/new/searchresult...
2220    https://www.cargurus.com/Cars/new/searchresult...
Name: urls, Length: 2221, dtype: object

In [141]:
cars_df = pd.DataFrame({})


In [133]:
failed_tries = []

In [136]:
start = 0
end = 200
# start with just 1 loop
for i in range(start,end):
    browser.visit(url_list[i])
    time.sleep(random.randint(3,6))
    
    # place into try / except block:
    try:
        # scrape using find by tag (section).  Should return list.
        keys = browser.find_by_tag('dl').find_by_tag('dt')
        values = browser.find_by_tag('dl').find_by_tag('dd')

        # loop through section list and find the one that has "Dealer's Price"
#         section_list = []
#         for e in elems:
#             section_list.append(e.html)
        # extract section list item into section variable

        # run replaces on tags in section to turn it into table
        

        # put table into single dataframe temp_df (NOT scrape_cars_df)
        temp_df = pd.DataFrame({})
        for j in range(len(keys)):

            temp_df[keys[j].text]= [values[j].text]
        temp_df
        
        # create search string to find remaining car data
        # search_str = " - " + temp_df["Dealer's Price:"][0]
        # find <h1> tag which contains search_str
        search_str = "IpF2YF"
        title = browser.find_by_tag('h1')
        price = title[0].html.split(' ')[-1]
        model = title[0].html.split(' ')[2]
        
        temp_df['Price:'] = [price]
        temp_df['Model:'] = [model]
        
        # split element string by space
        # year = e[0]
        # make = e[1]
        # model = e[2]
        dealernames = browser.find_by_tag('a')
        for d in dealernames:
            if d['data-cg-ft'] == 'vdp-dealer-cg-homepage-link':
                dealername = d.text
        temp_df['Dealer Name:'] = [dealername]
        # add columns and data to dataframe (temp_df['year'] = [year])
        dealeraddress = browser.find_by_tag('span')
        for d in dealeraddress:
            if d['data-track-ui'] == 'dealer-address':
                dealeradd = d.text
        dealeradd = dealeradd.replace('\n', '<br>')        
        temp_df["Dealer Address:"] = [dealeradd]
        # append new dataframe to scrape_cars_df if successful
        cars_df = pd.concat([cars_df, temp_df], axis=0, ignore_index=True)
        
        print(f"Adding {i} of {end}")
    
    # except: add i into failed list
    except:
        failed_tries.append(i)
cars_df.to_csv(f'../Resources/{start}-{end}.csv')

Adding 0 of 4
Adding 1 of 4
Adding 2 of 4


IndexError: list index out of range

In [117]:
dealeraddress = browser.find_by_tag('span')
for d in dealeraddress:
    if d['data-track-ui'] == 'dealer-address':
        dealeradd = d.text
dealeradd        

'2550 W University Dr\nProsper, TX 75078'

In [121]:
temp_df

,Make:,Model:,Year:,Trim:,Body type:,Exterior color:,Interior color:,VIN:,Stock number:,Price:,Dealer Name:,Dealer Address:
0,Jeep,Wrangler,2022,Sport 4WD,SUV / crossover,Bright White Clearcoat,Black,1C4GJXAN9NW272694,NW272694,"$33,485",Shottenkirk Prosper Chrysler Dodge Jeep Ram,"2550 W University Dr<br>Prosper, TX 75078"


In [138]:
cars_df

,Make:,Model:,Year:,Body type:,Exterior color:,Interior color:,Mileage:,VIN:,Stock number:,Price:,Dealer Name:,Dealer Address:,Trim:
0,Ford,Bronco,2022,SUV / crossover,Race Red,Red (Pv),15 mi,1FMDE5CP9NLB31549,221306,"$51,895",SouthWest Ford,"3000 Fort Worth Hwy<br>Weatherford, TX 76087",NaN
1,Ford,Bronco,2022,SUV / crossover,Cactus Gray,Gray (Tv),3 mi,1FMEE5DP4NLA73093,221307,"$59,565",SouthWest Ford,"3000 Fort Worth Hwy<br>Weatherford, TX 76087",NaN
2,Ford,Bronco,2022,SUV / crossover,Race Red,Red (Pv),15 mi,1FMDE5CP9NLB31549,221306,"$51,895",SouthWest Ford,"3000 Fort Worth Hwy<br>Weatherford, TX 76087",NaN
3,Ford,Bronco,2022,SUV / crossover,Cactus Gray,Gray (Tv),3 mi,1FMEE5DP4NLA73093,221307,"$59,565",SouthWest Ford,"3000 Fort Worth Hwy<br>Weatherford, TX 76087",NaN
4,Jeep,Wrangler,2022,SUV / crossover,Bright White,Black,0 mi,1C4HJXDG2NW227579,227579,"$48,270",Mac Haik Chrysler Dodge Jeep RAM Energy Corridor,"12111 Katy Fwy<br>Houston, TX 77079",High Tide 4WD


In [125]:
failed_tries

[]

In [73]:
search_str = "IpF2YF"
title = browser.find_by_tag('h1')
title[0].html.split(' ')[-1]

'$59,565'